In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beijing-hospital-pharmacy-sales-dataset/Beijing_hospital_pharmacy_sales_dataset.csv


# 1 理解数据

In [2]:
sales = pd.read_csv('../input/beijing-hospital-pharmacy-sales-dataset/Beijing_hospital_pharmacy_sales_dataset.csv')
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6578 entries, 0 to 6577
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   购药时间    6576 non-null   object 
 1   社保卡号    6576 non-null   float64
 2   商品编码    6577 non-null   float64
 3   商品名称    6577 non-null   object 
 4   销售数量    6577 non-null   float64
 5   应收金额    6577 non-null   float64
 6   实收金额    6577 non-null   float64
dtypes: float64(5), object(2)
memory usage: 359.9+ KB


In [3]:
sales.head()

,购药时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,1.616528e+06,236701.0,强力VC银翘片,6.0,82.8,69.00
1,2018-01-02 星期六,1.616528e+06,236701.0,清热解毒口服液,1.0,28.0,24.64
2,2018-01-06 星期三,1.260283e+07,236701.0,感康,2.0,16.8,15.00
3,2018-01-11 星期一,1.007034e+10,236701.0,三九感冒灵,1.0,28.0,28.00
4,2018-01-15 星期五,1.015543e+08,236701.0,三九感冒灵,8.0,224.0,208.00


# 2 数据清洗

In [4]:
#把购药时间改成销售时间比较有利于理解,inplace=True,直接在原数据框内改动
sales.rename(columns = {'购药时间':'销售时间'}, inplace = True)
#看一下有没有被改动
sales.head()

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,1.616528e+06,236701.0,强力VC银翘片,6.0,82.8,69.00
1,2018-01-02 星期六,1.616528e+06,236701.0,清热解毒口服液,1.0,28.0,24.64
2,2018-01-06 星期三,1.260283e+07,236701.0,感康,2.0,16.8,15.00
3,2018-01-11 星期一,1.007034e+10,236701.0,三九感冒灵,1.0,28.0,28.00
4,2018-01-15 星期五,1.015543e+08,236701.0,三九感冒灵,8.0,224.0,208.00


## 缺失值处理

In [5]:
# 没有时间和社保卡号的消费数据对于本次分析是无效的，
# 所以清理一下缺失值
sales = sales.dropna(subset = [ '销售时间','社保卡号'],how = 'any')
# sales

## 数据类型处理

In [6]:
sales['销售数量'] = sales['销售数量'].astype('float')
sales['应收金额'] = sales['应收金额'].astype('float')
sales['实收金额'] = sales['实收金额'].astype('float')

In [7]:
sales['销售时间'], sales['销售星期'] = sales['销售时间'].str.split(' ', 1).str

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [8]:
sales['销售时间'] = pd.to_datetime(sales['销售时间'], 
format = '%Y-%m-%d',errors = 'coerce')

## 排序

In [9]:
#对数据按照时间排序
sales = sales.sort_values('销售时间', ascending = True)
#排序之后再重置一下index
sales = sales.reset_index(drop = True)
sales.head()

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额,销售星期
0,2018-01-01,1.616528e+06,236701.0,强力VC银翘片,6.0,82.8,69.0,星期五
1,2018-01-01,1.078916e+08,861456.0,酒石酸美托洛尔片(倍他乐克),2.0,14.0,12.6,星期五
2,2018-01-01,1.616528e+06,861417.0,雷米普利片(瑞素坦),1.0,28.5,28.5,星期五
3,2018-01-01,1.007397e+10,866634.0,硝苯地平控释片(欣然),6.0,111.0,92.5,星期五
4,2018-01-01,1.001429e+10,866851.0,缬沙坦分散片(易达乐),1.0,26.0,23.0,星期五


## 异常值处理

In [10]:
sales.describe()

,社保卡号,商品编码,销售数量,应收金额,实收金额
count,6.575000e+03,6.575000e+03,6575.000000,6575.000000,6575.000000
mean,6.092179e+09,1.015458e+06,2.385095,50.478935,46.321582
std,4.889081e+09,5.126518e+05,2.373702,87.607883,80.987682
min,1.616528e+06,2.367010e+05,-10.000000,-374.000000,-374.000000
25%,1.014234e+08,8.614560e+05,1.000000,14.000000,12.320000
50%,1.001650e+10,8.615070e+05,2.000000,28.000000,26.600000
75%,1.004882e+10,8.689265e+05,2.000000,59.600000,53.000000
max,1.283612e+10,2.367012e+06,50.000000,2950.000000,2650.000000


In [11]:
#选取销售数量和应收金额大于0的列
sales = sales[(sales['销售数量'] > 0) & (sales['应收金额'] > 0)]

# 3 数据分析

## 月均消费次数

In [12]:
#首先对数据进行一个去重，使用drop_duplicates函数
salesqc = sales.drop_duplicates(subset = ['销售时间', '社保卡号'])
#去重后看一下共有多少条购买数据(结果为5363)
Total = salesqc.shape[0]

#再计算月份
#用销售时间的最大值减去最小值即可得到天数,再除以（地板除）三十就可以得到月份了
month = (sales['销售时间'].max() - sales['销售时间'].min()).days//30

KPI1 = Total / month
print('月均消费次数为：', KPI1)

月均消费次数为： 893.8333333333334


## 月均消费金额

In [13]:
#计算总金额
Total1 = sales['实收金额'].sum()

KPI2 = Total1 / month

print('月均消费金额为：', KPI2)

月均消费金额为： 50909.695


## 客单价

In [14]:
kdj = Total1 / Total

print('客单价为：', kdj)

客单价为： 56.9565858661197


## 消费趋势

In [15]:
##对去重后的数据按照天进行重新采样
#首先要把索引变成时间
sales.index = pd.DatetimeIndex(sales['销售时间'])
#然后对其按照每天从新采样
salesd = sales.resample('D').count()

In [16]:
#画图
# salesd.plot(x = salesd.index, y = '实收金额')
# plt.xlabel('Time')
# plt.ylabel('Money')
# plt.title('销售数据')
# plt.show()

In [17]:
#按月采样
# salesm = sales.resample('M').count()
# #画图
# salesm.plot(x = salesm.index, y = '实收金额')
# plt.xlabel('Time')
# plt.ylabel('People')
# plt.title('ke liu liang (month)')
# plt.show()

In [18]:
sales.groupby('销售星期')['实收金额'].sum()

销售星期
星期一    40354.99
星期三    39073.64
星期二    44534.63
星期五    49817.87
星期六    47834.33
星期四    40310.08
星期日    43532.63
Name: 实收金额, dtype: float64